In [1]:
%%capture
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

In [2]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-07-09 05:21:04.180790: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 05:21:04.180942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 05:21:04.475906: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from huggingface_hub import login
login('')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
! pip install accelerate peft bitsandbytes transformers trl sentencepiece wandb

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [5]:
from datasets import load_dataset, concatenate_datasets
import random
from datasets import Dataset, DatasetDict


dataset = load_dataset("Angelectronic/ViMMRC")
dataset

Generating train split:   0%|          | 0/1936 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/551 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/296 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'options', 'answers'],
        num_rows: 1936
    })
    test: Dataset({
        features: ['context', 'question', 'options', 'answers'],
        num_rows: 551
    })
    dev: Dataset({
        features: ['context', 'question', 'options', 'answers'],
        num_rows: 296
    })
})

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-7b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = '',
)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # 4x longer contexts auto supported!
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
tokenizer.padding_side = "left"

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Unsloth 2024.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [7]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
END_TURN_TOKEN = '<end_of_turn>'
BOS_TOKEN = tokenizer.bos_token

def formatting_prompts_func(examples):
    texts = []
    for i in range(len(examples['question'])):
        option_prompt = ""
        for j in range(len(examples['options'][i])):
            if j == 0:
                option_prompt += "A. " + examples['options'][i][j] + "\n"
            elif j == 1:
                option_prompt += "B. " + examples['options'][i][j] + "\n"
            elif j == 2:
                option_prompt += "C. " + examples['options'][i][j] + "\n"
            elif j == 3:
                option_prompt += "D. " + examples['options'][i][j] + "\n"

        chat = [
            {"role": "user", "content": "Use the following reference to answer the given multiple choice-question:\nReference: " + examples['context'][i] + "\nQuestion: " + examples['question'][i] + "\n" + option_prompt},
            {"role": "assistant", "content": examples['answers'][i]}
        ]
        texts.append(tokenizer.apply_chat_template(chat, tokenize=False) + EOS_TOKEN)

    texts = [text.split(BOS_TOKEN)[-1] for text in texts]
    return {"text": texts}
pass
dataset['train'] = dataset['train'].map(formatting_prompts_func, batched=True)
dataset['test'] = dataset['test'].map(formatting_prompts_func, batched=True)

collator = DataCollatorForCompletionOnlyLM(instruction_template="<|im_start|>user", response_template="<|im_start|>assistant", tokenizer=tokenizer, mlm=False)

print(dataset['train'][0]['text'])

Map:   0%|          | 0/1936 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]

<|im_start|>user
Use the following reference to answer the given multiple choice-question:
Reference: Người mẹ đang bận rộn nấu bữa tối trong bếp, bất ngờ câu con trai bé bỏng chạy ùa vào và đưa cho mẹ một mẩu giấy nhỏ. Sau khi lau tay vào chiếc tạp dề, người mẹ mở tờ giấy ra và đọc:

Cắt cỏ trong vườn - 5 đô la
Dọn dẹp phòng của con - 1 đô la
Đi chợ cùng với mẹ - 50 xu
Trông em giúp mẹ - 25 xu
Đổ rác - 1 đô la
Kết quả học tập tốt - 5 đô la
Quét dọn sân - 2 đô la
Mẹ nợ con tổng cộng - 14,75 đôla

Sau khi đọc xong, người mẹ nhìn cậu con trai dang đứng chờ với vẻ mặt đầy hi vọng. Bà cầm bút lên, lật mặt sau của tờ giấy và viết:

Chín tháng mười ngày con nằm trong bụng mẹ: Miễn phí.
Những lúc mẹ ở bên cạnh chăm sóc, cầu nguyện mỗi khi con ốm đau: Miễn phí.
Những giọt nước mắt của con làm mẹ khóc trong những năm qua: Miễn phí.
Những đêm mẹ không ngủ vì lo lắng cho tương lai của con: Miễn phí.
Tất cả những đồ chơi, thức ăn, quần áo mà mẹ đã nuôi con trong suốt mấy năm qua: Miễn phí.
Và đắt 

In [8]:
from transformers.integrations import WandbCallback
from transformers import TrainerCallback
from tqdm import tqdm
import os
import pandas as pd
import wandb
from sklearn.metrics import accuracy_score

wandb.login(key="fe5c6aaeafc7ad17e6dd649c9e4595f331ce5b94")

os.environ["WANDB_PROJECT"] = "iai_llm_base_model"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

class WandbPredictionProgressCallback(TrainerCallback):
    def __init__(self, trainer, tokenizer, val_dataset, num_samples=551):
        super().__init__()
        self.trainer = trainer
        self.tokenizer = tokenizer
        self.sample_dataset = val_dataset.select(range(num_samples))
        # self.table = self._wandb.Table(columns=["step", "score"])

    def on_evaluate(self, args, state, control, **kwargs):
        labels = [self.sample_dataset[i]['answers'] for i in range(len(self.sample_dataset))]
        preds = []

        # choose a random sample from the validation dataset

        for i in tqdm(range(len(self.sample_dataset))):
            option_prompt = ""
            for j in range(len(self.sample_dataset[i]['options'])):
                if j == 0:
                    option_prompt += "A. " + self.sample_dataset[i]['options'][j] + "\n"
                elif j == 1:
                    option_prompt += "B. " + self.sample_dataset[i]['options'][j] + "\n"
                elif j == 2:
                    option_prompt += "C. " + self.sample_dataset[i]['options'][j] + "\n"
                elif j == 3:
                    option_prompt += "D. " + self.sample_dataset[i]['options'][j] + "\n"

            chat = [
                {"role": "user", "content": "Use the following reference to answer the given multiple choice-question:\nReference: " + self.sample_dataset[i]['context'] + "\nQuestion: " + self.sample_dataset[i]['question'] + "\n" + option_prompt},
            ]

            prompt = tokenizer.apply_chat_template(chat, tokenize=False,add_generation_prompt=True)
            prompt = prompt.split(BOS_TOKEN)[-1]
            inputs = tokenizer(
                prompt,
                return_tensors = "pt",
            ).to("cuda")

            # generate without additional prompt
            outputs = self.trainer.model.generate(**inputs, max_new_tokens = 256, use_cache = True, eos_token_id=self.trainer.model.config.eos_token_id, pad_token_id=tokenizer.eos_token_id)

            prediction = tokenizer.decode(outputs[0], skip_special_tokens = False)
            completion = prediction.split('<|im_start|>assistant')[-1].split(END_TURN_TOKEN)[0].strip().split(EOS_TOKEN)[0].strip()
            preds.append(completion)

        print("step: ", state.global_step, "accuracy: ", accuracy_score(labels, preds))

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
from transformers import TrainingArguments
import re
import numpy as np

random_indices_test = random.sample(range(len(dataset['test'])), 551)
test_set = dataset['test'].select(random_indices_test)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = test_set,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    data_collator=collator,               # not good with instruct model
    args = TrainingArguments(
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps = 2,
        eval_accumulation_steps=2,
        warmup_steps = 5,  # change logging step too
        num_train_epochs=3,
        learning_rate = 5e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,   ####
        save_steps=10,   ####
        optim = "adamw_8bit",
        report_to=["tensorboard"],
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "/gemma-base-ViMMRC",
        push_to_hub = True,
        evaluation_strategy='steps',
        load_best_model_at_end=True,
        max_grad_norm=0.5,
    ),
)

progress_callback = WandbPredictionProgressCallback(trainer, tokenizer, dataset['test'], num_samples=200)
trainer.add_callback(progress_callback)

trainer_stats = trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/1936 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/551 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,936 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 2,904
 "-____-"     Number of trainable parameters = 50,003,968


Step,Training Loss,Validation Loss
10,0.924700,0.078189


  0%|          | 0/200 [00:00<?, ?it/s]


KeyError: 'Cache only has 0 layers, attempted to access layer with index 0'